<a href="https://colab.research.google.com/github/EDC21/Deep-Learning/blob/master/Reusing_Pretrained_Layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Reusing Pretrained Layers

It is generally not a good idea to train a very large DNN from scratch: insteard, you should always try to find an existing neural network that accomplishes a similar task to the one you are trying to tackle, then reuse the lower layers of this network.

##Transfer Learning

Example:  
Let's split the fashion MNIST training set in two:
* `X_train_A`: all images of all items except for sandals and shirts (classes 5 and 6).
* `X_train_B`: a much smaller training set of just the first 200 images of sandals or shirts.

The validation set and the test set are also split this way, but without restricting the number of images.

We will train a model on set A (classification task with 8 classes), and try to reuse it to tackle set B (binary classification). We hope to transfer a little bit of knowledge from task A to task B, since classes in set A (sneakers, ankle boots, coats, t-shirts, etc.) are somewhat similar to classes in set B (sandals and shirts). However, since we are using `Dense` layers, only patterns that occur at the same location can be reused (in contrast, convolutional layers will transfer much better, since learned patterns can be detected anywhere on the image, as we will see in the CNN chapter).


In [13]:
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)



The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


##Data Manipulation

In [6]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

4423680/4422102 [==============================] - 0s 0us/step


In [7]:
X_train_A.shape

(43986, 28, 28)

In [8]:
X_train_B.shape

(200, 28, 28)

In [9]:
y_train_A[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [10]:
y_train_B[:30]

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

##Model A Architecture

Build model_A (input=X_train_A)

In [14]:
tf.random.set_seed(42)
np.random.seed(42)

In [15]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [16]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [17]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.5926 - accuracy: 0.8103 - val_loss: 0.3890 - val_accuracy: 0.8677
Epoch 2/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.3523 - accuracy: 0.8786 - val_loss: 0.3289 - val_accuracy: 0.8824
Epoch 3/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.3170 - accuracy: 0.8895 - val_loss: 0.3013 - val_accuracy: 0.8991
Epoch 4/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.2972 - accuracy: 0.8974 - val_loss: 0.2892 - val_accuracy: 0.9013
Epoch 5/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.2834 - accuracy: 0.9022 - val_loss: 0.2774 - val_accuracy: 0.9066
Epoch 6/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.2729 - accuracy: 0.9063 - val_loss: 0.2733 - val_accuracy: 0.9071
Epoch 7/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.2641 - accuracy: 0.9092 - val_loss: 0.2720 - val_accuracy:

In [18]:
model_A.save("my_model_A.h5")

##Model B Architecture

Build model_B (input=X_train_B)

In [21]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [22]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [23]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 0s 27ms/step - loss: 0.5486 - accuracy: 0.7300 - val_loss: 0.4821 - val_accuracy: 0.8012
Epoch 2/20
7/7 [==============================] - 0s 10ms/step - loss: 0.3941 - accuracy: 0.8550 - val_loss: 0.3780 - val_accuracy: 0.8682
Epoch 3/20
7/7 [==============================] - 0s 11ms/step - loss: 0.3016 - accuracy: 0.9150 - val_loss: 0.3135 - val_accuracy: 0.9087
Epoch 4/20
7/7 [==============================] - 0s 10ms/step - loss: 0.2448 - accuracy: 0.9600 - val_loss: 0.2684 - val_accuracy: 0.9260
Epoch 5/20
7/7 [==============================] - 0s 10ms/step - loss: 0.2043 - accuracy: 0.9750 - val_loss: 0.2376 - val_accuracy: 0.9391
Epoch 6/20
7/7 [==============================] - 0s 10ms/step - loss: 0.1761 - accuracy: 0.9800 - val_loss: 0.2146 - val_accuracy: 0.9483
Epoch 7/20
7/7 [==============================] - 0s 10ms/step - loss: 0.1553 - accuracy: 0.9800 - val_loss: 0.1972 - val_accuracy: 0.9533
Epoch 8/20
7/7 [===========

In [25]:
model_B.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_13 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_14 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_15 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_16 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_17 (Dense)             (None, 1)                

##Load model A and create a new model based on that model's layers

Build model_B_on_A

In [27]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1,activation="sigmoid"))

Note that model_A and model_B_on_A now shares some layers. When we train model_B_on_A, it will also affect model_A. To avoid that, we need to clone model_A before we reuse its layers.  
To do this, we clone model A's architecture with clone.model(), then copy its weights.

In [28]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [34]:
for layer in model_B_on_A.layers[:-1]:
  layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
          optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])

In [35]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
              validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
          optimizer=keras.optimizers.SGD(lr=1e-3),
          metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
              validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 0s 28ms/step - loss: 0.4481 - accuracy: 0.8350 - val_loss: 0.4436 - val_accuracy: 0.8174
Epoch 2/4
7/7 [==============================] - 0s 9ms/step - loss: 0.4157 - accuracy: 0.8500 - val_loss: 0.4145 - val_accuracy: 0.8306
Epoch 3/4
7/7 [==============================] - 0s 10ms/step - loss: 0.3870 - accuracy: 0.8600 - val_loss: 0.3880 - val_accuracy: 0.8458
Epoch 4/4
7/7 [==============================] - 0s 9ms/step - loss: 0.3610 - accuracy: 0.8700 - val_loss: 0.3650 - val_accuracy: 0.8529
Epoch 1/16
7/7 [==============================] - 0s 27ms/step - loss: 0.2943 - accuracy: 0.9250 - val_loss: 0.2487 - val_accuracy: 0.9523
Epoch 2/16
7/7 [==============================] - 0s 11ms/step - loss: 0.2061 - accuracy: 0.9850 - val_loss: 0.1968 - val_accuracy: 0.9797
Epoch 3/16
7/7 [==============================] - 0s 11ms/step - loss: 0.1635 - accuracy: 0.9900 - val_loss: 0.1655 - val_accuracy: 0.9899
Epoch 4/16
7/7 [=================

In [36]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0931 - accuracy: 0.9835


[0.09306484460830688, 0.9835000038146973]

In [37]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0635 - accuracy: 0.9980


[0.06347920745611191, 0.9980000257492065]

Great! We got quite a bit of transfer: the error rate dropped by a fatcor of 4!

In [38]:
(100 - 96.95) / (100 - 99.25)

4.066666666666663